### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# import all relevant packages

import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.cross_validation import train_test_split


%matplotlib inline

/anaconda3/envs/dsi/lib/python3.6/site-packages/pandas_profiling/plot.py:15: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/anaconda3/envs/dsi/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/anaconda3/envs/dsi/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/anaconda3/envs/dsi/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda3/envs/dsi/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop

In [3]:
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score


### Read CSV

In [4]:
train = pd.read_csv('./assets/train.csv')
test = pd.read_csv('./assets/test.csv')
weather = pd.read_csv('./assets/weather.csv')
spray = pd.read_csv('./assets/spray.csv')

##### Check Data Types and Columns name

**from weather descr document**
- `M`: Missing Data, or data that are not normally reported for the station
- `-`: values not available for sunrise/sunset.
    - because each date has two reported data from station 1 and 2. Therefore, station 2 column of the date has no sunrise/sunset data.

##### dtypes

In [5]:
print("Train set data types:\n")
print(train.dtypes)
print("-----")
print("Test set data types:\n")
print(test.dtypes)
print("-----")
print("Weather set data types:\n")
print(weather.dtypes)
print("-----")
print("Spray set data types:\n")
print(spray.dtypes)


Train set data types:

Date                       object
Address                    object
Species                    object
Block                       int64
Street                     object
Trap                       object
AddressNumberAndStreet     object
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
NumMosquitos                int64
WnvPresent                  int64
dtype: object
-----
Test set data types:

Id                          int64
Date                       object
Address                    object
Species                    object
Block                       int64
Street                     object
Trap                       object
AddressNumberAndStreet     object
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
dtype: object
-----
Weather set data types:

Station          int64
Date            object
Tmax             int64
Tmin             int64
Tavg            

In [6]:
# only select `object` columns
train_cats = train.select_dtypes(include=['object']).columns
test_cats = test.select_dtypes(include=['object']).columns
weather_cats = weather.select_dtypes(include=['object']).columns
spray_cats = spray.select_dtypes(include=['object']).columns

print(train_cats,'\n')
print(test_cats,'\n')
print(weather_cats,'\n')
print(spray_cats,'\n')

Index(['Date', 'Address', 'Species', 'Street', 'Trap',
       'AddressNumberAndStreet'],
      dtype='object') 

Index(['Date', 'Address', 'Species', 'Street', 'Trap',
       'AddressNumberAndStreet'],
      dtype='object') 

Index(['Date', 'Tavg', 'Depart', 'WetBulb', 'Heat', 'Cool', 'Sunrise',
       'Sunset', 'CodeSum', 'Depth', 'Water1', 'SnowFall', 'PrecipTotal',
       'StnPressure', 'SeaLevel', 'AvgSpeed'],
      dtype='object') 

Index(['Date', 'Time'], dtype='object') 



##### Weather Dataset

In [7]:
weather.head(5).T

,0,1,2,3,4
Station,1,2,1,2,1
Date,2007-05-01,2007-05-01,2007-05-02,2007-05-02,2007-05-03
Tmax,83,84,59,60,66
Tmin,50,52,42,43,46
Tavg,67,68,51,52,56
Depart,14,M,-3,M,2
DewPoint,51,51,42,42,40
WetBulb,56,57,47,47,48
Heat,0,0,14,13,9
Cool,2,3,0,0,0


In [8]:
weather.describe()

,Station,Tmax,Tmin,DewPoint,ResultSpeed,ResultDir
count,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000,2944.000000
mean,1.500000,76.166101,57.810462,53.457880,6.960666,17.494905
std,0.500085,11.461970,10.381939,10.675181,3.587527,10.063609
min,1.000000,41.000000,29.000000,22.000000,0.100000,1.000000
25%,1.000000,69.000000,50.000000,46.000000,4.300000,7.000000
50%,1.500000,78.000000,59.000000,54.000000,6.400000,19.000000
75%,2.000000,85.000000,66.000000,62.000000,9.200000,25.000000
max,2.000000,104.000000,83.000000,75.000000,24.100000,36.000000


In [9]:
weather['Station'].nunique()

2

##### Found some suspicious data
- 11 missing values in `weather['Tavg']` (should've `int`)
- 1472 values missing in `weather['Depart']`

In [10]:
(weather[weather.columns] == 'M').sum().sort_values(ascending=False)

Water1         2944
Depart         1472
SnowFall       1472
Depth          1472
Tavg             11
Cool             11
Heat             11
SeaLevel          9
StnPressure       4
WetBulb           4
AvgSpeed          3
PrecipTotal       2
Date              0
Tmax              0
Tmin              0
Sunrise           0
DewPoint          0
ResultDir         0
Sunset            0
CodeSum           0
ResultSpeed       0
Station           0
dtype: int64

In [11]:
weather[weather['Tavg'] == 'M']

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
7,2,2007-05-04,78,51,M,M,42,50,M,M,...,,M,M,M,0.00,29.36,30.04,10.1,7,10.4
505,2,2008-07-08,86,46,M,M,68,71,M,M,...,TS RA,M,M,M,0.28,29.16,29.80,7.4,24,8.3
675,2,2008-10-01,62,46,M,M,41,47,M,M,...,,M,M,M,0.00,29.30,29.96,10.9,33,11.0
1637,2,2011-07-22,100,71,M,M,70,74,M,M,...,TS TSRA BR,M,M,M,0.14,29.23,29.86,3.8,10,8.2
2067,2,2012-08-22,84,72,M,M,51,61,M,M,...,,M,M,M,0.00,29.39,M,4.7,19,M
2211,2,2013-05-02,71,42,M,M,39,45,M,M,...,,M,M,M,0.00,29.51,30.17,15.8,2,16.1
2501,2,2013-09-24,91,52,M,M,48,54,M,M,...,,M,M,M,0.00,29.33,30.00,5.8,9,7.7
2511,2,2013-09-29,84,53,M,M,48,54,M,M,...,RA BR,M,M,M,0.22,29.36,30.01,6.3,36,7.8
2525,2,2013-10-06,76,48,M,M,44,50,M,M,...,RA DZ BR,M,M,M,0.06,29.10,29.76,10.1,25,10.6
2579,2,2014-05-02,80,47,M,M,43,47,M,M,...,RA,M,M,M,0.04,29.10,29.79,10.7,23,11.9


In [12]:
weather['Depart'].value_counts().head()

M     1472
 2      93
-1      84
-2      80
 5      77
Name: Depart, dtype: int64

In [13]:
print(weather['WetBulb'].nunique())
weather['WetBulb'].value_counts().head()

48


63    135
65    131
59    129
61    123
64    121
Name: WetBulb, dtype: int64

In [14]:
print(weather['Heat'].nunique())
weather['Heat'].value_counts().head()

31


0    1870
4      88
1      86
2      81
8      67
Name: Heat, dtype: int64

In [15]:
print(weather['Cool'].nunique())
weather['Cool'].value_counts().head()

31


 0    1147
 8     138
 5     117
12     117
10     110
Name: Cool, dtype: int64

In [16]:
weather['Depth'].value_counts()

0    1472
M    1472
Name: Depth, dtype: int64

In [17]:
# Since all the data is missing. we can omit!
weather['Water1'].value_counts()

M    2944
Name: Water1, dtype: int64

In [18]:
# It also seems irrelavant. no meaningful values
weather['SnowFall'].value_counts()

M      1472
0.0    1459
  T      12
0.1       1
Name: SnowFall, dtype: int64

In [19]:
# Given Water1 contains all missing values, and depth and snowfall also do not contain any useful information
# I'm going to go ahead and drop these columns 

In [20]:
weather= weather.drop(['Depth','SnowFall','Water1'], axis=1)

##### Can we extract "CodeSum"?

In [21]:
# can we break it by Regex?

weather['CodeSum'].value_counts()

                        1609
RA                       296
RA BR                    238
BR                       110
TSRA RA BR                92
BR HZ                     81
RA DZ BR                  65
TSRA RA                   43
HZ                        39
RA BR HZ                  38
TSRA                      34
RA DZ                     22
TSRA BR                   21
TS TSRA RA BR             19
RA HZ                     16
TS RA                     13
TSRA RA BR HZ             12
TS                        10
TS TSRA BR                10
DZ BR                     10
DZ BR HZ                   9
TS RA BR                   8
TSRA BR HZ                 8
DZ                         8
RA DZ BR HZ                7
TS TSRA BR HZ              7
TS TSRA                    7
TS TSRA RA                 7
TSRA RA BR VCTS            4
TS BR                      4
                        ... 
FG+ MIFG BR                1
TSRA RA FG+ FG BR HZ       1
TS TSRA RA FG BR HZ        1
TS TSRA VCFG  

In [22]:
# NEW one from Brian Collins
patterns = ['SH', 'DZ', 'RA', 'TS', 'BR']

# iterate through all the CodeSum to find the columns with the patterns list
weather['CodeSum'] = weather['CodeSum'].apply(lambda x: ' '.join([t for t in x.split(' ') 
                                                                   if t in patterns]))

weather.CodeSum.value_counts()

            1692
RA BR        406
RA           366
BR           245
RA DZ BR      81
TS RA BR      37
TS BR         23
RA DZ         23
TS RA         22
DZ BR         21
TS            20
DZ             8
Name: CodeSum, dtype: int64

Let's keep take a look on other columns

In [23]:
# 318 values are just trace rainfall

weather.PrecipTotal=weather.PrecipTotal.str.strip()
weather[weather.PrecipTotal == 'T']

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
6,1,2007-05-04,66,49,58,4,41,50,7,0,0444,1852,RA,T,29.31,30.05,10.4,8,10.8
8,1,2007-05-05,66,53,60,5,38,49,5,0,0443,1853,,T,29.40,30.10,11.7,7,12.0
9,2,2007-05-05,66,54,60,M,39,50,5,0,-,-,,T,29.46,30.09,11.2,7,11.5
12,1,2007-05-07,83,47,65,10,41,54,0,0,0441,1856,RA,T,29.38,30.12,8.6,18,10.5
15,2,2007-05-08,80,60,70,M,57,63,0,5,-,-,,T,29.36,30.02,2.5,8,5.4
47,2,2007-05-24,88,66,77,M,53,63,0,12,-,-,RA,T,29.47,30.10,13.8,20,15.2
48,1,2007-05-25,70,56,63,1,44,53,2,0,0423,1914,,T,29.51,30.24,4.9,5,8.8
54,1,2007-05-28,79,50,65,2,47,56,0,0,0421,1916,RA,T,29.43,30.15,4.8,10,5.3
65,2,2007-06-02,81,68,75,M,62,67,0,10,-,-,RA BR,T,29.10,29.76,7.5,21,8.2
73,2,2007-06-06,73,48,61,M,47,55,4,0,-,-,RA,T,29.13,29.82,9.6,15,10.6


In [24]:
# 0.00 rainfall - 1,577 values
weather[weather.PrecipTotal == '0.00']

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,0448,1849,,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,-,-,,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,0447,1850,BR,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,-,-,BR,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,0446,1851,,0.00,29.39,30.12,11.7,7,11.9
5,2,2007-05-03,67,48,58,M,40,50,7,0,-,-,,0.00,29.46,30.12,12.9,6,13.2
7,2,2007-05-04,78,51,M,M,42,50,M,M,-,-,,0.00,29.36,30.04,10.1,7,10.4
10,1,2007-05-06,68,49,59,4,30,46,6,0,0442,1855,,0.00,29.57,30.29,14.4,11,15.0
11,2,2007-05-06,68,52,60,M,30,46,5,0,-,-,,0.00,29.62,30.28,13.8,10,14.5
13,2,2007-05-07,84,50,67,M,39,53,0,2,-,-,,0.00,29.44,30.12,8.5,17,9.9


In [25]:
weather.groupby('Station')['Tmax'].mean()

Station
1    75.978940
2    76.353261
Name: Tmax, dtype: float64

##### Split station 1 and 2, join horizontally (courtesy of abhishek of kaggle)

In [26]:
weather_stn1 = weather[weather['Station'] == 1]
weather_stn2 = weather[weather['Station'] == 2]
weather_stn1 = weather_stn1.drop('Station', axis=1)
weather_stn2 = weather_stn2.drop('Station', axis=1)
weather = weather_stn1.merge(weather_stn2, on='Date')

In [27]:
(weather[weather.columns] == 'M').sum().sort_values(ascending=False)

Depart_y         1472
Cool_y             11
Tavg_y             11
Heat_y             11
SeaLevel_x          5
SeaLevel_y          4
AvgSpeed_y          3
WetBulb_x           3
StnPressure_x       2
PrecipTotal_y       2
StnPressure_y       2
WetBulb_y           1
Depart_x            0
Tavg_x              0
CodeSum_x           0
DewPoint_x          0
Tmin_x              0
Heat_x              0
Cool_x              0
Sunrise_x           0
Sunset_x            0
Tmax_x              0
AvgSpeed_x          0
PrecipTotal_x       0
ResultSpeed_x       0
ResultDir_x         0
ResultDir_y         0
Tmax_y              0
Tmin_y              0
DewPoint_y          0
Sunrise_y           0
Sunset_y            0
CodeSum_y           0
ResultSpeed_y       0
Date                0
dtype: int64

In [28]:
# replace missing values and T with -1 (courtesy of abhishek of kaggle)

weather = weather.replace("M", -1)
weather = weather.replace("-", -1)
weather = weather.replace("T", -1)
weather = weather.replace(" T", -1)
weather = weather.replace("  T", -1)

In [29]:
weather.describe()

,Tmax_x,Tmin_x,DewPoint_x,ResultSpeed_x,ResultDir_x,Tmax_y,Tmin_y,Depart_y,DewPoint_y,Sunrise_y,Sunset_y,ResultSpeed_y,ResultDir_y
count,1472.00000,1472.000000,1472.000000,1472.000000,1472.000000,1472.000000,1472.000000,1472.0,1472.000000,1472.0,1472.0,1472.000000,1472.00000
mean,75.97894,56.730978,53.377717,6.987092,17.656250,76.353261,58.889946,-1.0,53.538043,-1.0,-1.0,6.934239,17.33356
std,11.50315,10.347907,10.670263,3.586353,10.222077,11.421482,10.306855,0.0,10.683121,0.0,0.0,3.589725,9.90345
min,42.00000,29.000000,22.000000,0.100000,1.000000,41.000000,29.000000,-1.0,23.000000,-1.0,-1.0,0.300000,1.00000
25%,68.00000,49.000000,46.000000,4.400000,7.000000,69.000000,51.000000,-1.0,46.000000,-1.0,-1.0,4.275000,8.00000
50%,78.00000,58.000000,54.000000,6.400000,19.500000,78.000000,60.000000,-1.0,55.000000,-1.0,-1.0,6.500000,19.00000
75%,85.00000,65.000000,61.250000,9.100000,26.000000,85.000000,67.000000,-1.0,62.000000,-1.0,-1.0,9.225000,24.00000
max,103.00000,82.000000,75.000000,24.100000,36.000000,104.000000,83.000000,-1.0,74.000000,-1.0,-1.0,21.700000,36.00000


In [30]:
# drop the columns with all missing values
weather.drop(columns=['Depart_y', 'Sunrise_y', 'Sunset_y'], axis=1, inplace=True)
weather.describe()

,Tmax_x,Tmin_x,DewPoint_x,ResultSpeed_x,ResultDir_x,Tmax_y,Tmin_y,DewPoint_y,ResultSpeed_y,ResultDir_y
count,1472.00000,1472.000000,1472.000000,1472.000000,1472.000000,1472.000000,1472.000000,1472.000000,1472.000000,1472.00000
mean,75.97894,56.730978,53.377717,6.987092,17.656250,76.353261,58.889946,53.538043,6.934239,17.33356
std,11.50315,10.347907,10.670263,3.586353,10.222077,11.421482,10.306855,10.683121,3.589725,9.90345
min,42.00000,29.000000,22.000000,0.100000,1.000000,41.000000,29.000000,23.000000,0.300000,1.00000
25%,68.00000,49.000000,46.000000,4.400000,7.000000,69.000000,51.000000,46.000000,4.275000,8.00000
50%,78.00000,58.000000,54.000000,6.400000,19.500000,78.000000,60.000000,55.000000,6.500000,19.00000
75%,85.00000,65.000000,61.250000,9.100000,26.000000,85.000000,67.000000,62.000000,9.225000,24.00000
max,103.00000,82.000000,75.000000,24.100000,36.000000,104.000000,83.000000,74.000000,21.700000,36.00000


In [31]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1472 entries, 0 to 1471
Data columns (total 32 columns):
Date             1472 non-null object
Tmax_x           1472 non-null int64
Tmin_x           1472 non-null int64
Tavg_x           1472 non-null object
Depart_x         1472 non-null object
DewPoint_x       1472 non-null int64
WetBulb_x        1472 non-null object
Heat_x           1472 non-null object
Cool_x           1472 non-null object
Sunrise_x        1472 non-null object
Sunset_x         1472 non-null object
CodeSum_x        1472 non-null object
PrecipTotal_x    1472 non-null object
StnPressure_x    1472 non-null object
SeaLevel_x       1472 non-null object
ResultSpeed_x    1472 non-null float64
ResultDir_x      1472 non-null int64
AvgSpeed_x       1472 non-null object
Tmax_y           1472 non-null int64
Tmin_y           1472 non-null int64
Tavg_y           1472 non-null object
DewPoint_y       1472 non-null int64
WetBulb_y        1472 non-null object
Heat_y           1472 non

In [32]:
# seems very redundant code line, anyone can make it simpler way by creating function?

weather['Tavg_x']= weather.Tavg_x.astype(int)
weather['Tavg_y']= weather.Tavg_y.astype(int)
weather['AvgSpeed_x']=weather.AvgSpeed_x.astype(float)
weather['AvgSpeed_y']=weather.AvgSpeed_y.astype(float)
weather['Heat_x']=weather.Heat_x.astype(int)
weather['Heat_y']=weather.Heat_y.astype(int)
weather['Cool_x']=weather.Cool_x.astype(int)
weather['Cool_y']=weather.Cool_y.astype(int)
weather['PrecipTotal_x']=weather.PrecipTotal_x.astype(float)
weather['PrecipTotal_y']=weather.PrecipTotal_y.astype(float)
weather['StnPressure_x']=weather.StnPressure_x.astype(float)
weather['StnPressure_y']=weather.StnPressure_y.astype(float)
weather['StnPressure_x']=weather.StnPressure_x.astype(float)
weather['StnPressure_y']=weather.StnPressure_y.astype(float)
weather['SeaLevel_x']=weather.SeaLevel_x.astype(float)
weather['SeaLevel_y']=weather.SeaLevel_y.astype(float)

**Date Separate**

In [33]:
# courtesy of Brian Collins

def date_separate(df):
    df = df.copy()
    df['Year'] = pd.DatetimeIndex(df['Date']).year
    df['Month'] = pd.DatetimeIndex(df['Date']).month
    df['Day'] = pd.DatetimeIndex(df['Date']).day
    return df

# make all the dataset applied to the function
train=date_separate(train)
test=date_separate(test)
weather=date_separate(weather)
spray=date_separate(spray)

##### Check Train and Test set

In [34]:
train.head(1)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Year,Month,Day
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,2007,5,29


In [35]:
train.describe()

,Block,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Year,Month,Day
count,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000
mean,35.687797,41.841139,-87.699908,7.819532,12.853512,0.052446,2009.576242,7.692557,15.341424
std,24.339468,0.112742,0.096514,1.452921,16.133816,0.222936,2.354645,1.067675,8.901205
min,10.000000,41.644612,-87.930995,3.000000,1.000000,0.000000,2007.000000,5.000000,1.000000
25%,12.000000,41.732984,-87.760070,8.000000,2.000000,0.000000,2007.000000,7.000000,7.000000
50%,33.000000,41.846283,-87.694991,8.000000,5.000000,0.000000,2009.000000,8.000000,15.000000
75%,52.000000,41.954690,-87.627796,9.000000,17.000000,0.000000,2011.000000,8.000000,24.000000
max,98.000000,42.017430,-87.531635,9.000000,50.000000,1.000000,2013.000000,10.000000,31.000000


In [36]:
# Checking duplicates rows
# however it has different date, so we decide to keep
print(train.duplicated().value_counts())
train[train.duplicated()]

False    9693
True      813
dtype: int64


,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Year,Month,Day
99,2007-06-26,"2200 West 113th Street, Chicago, IL 60643, USA",CULEX PIPIENS/RESTUANS,22,W 113TH ST,T086,"2200 W 113TH ST, Chicago, IL",41.688324,-87.676709,8,1,0,2007,6,26
295,2007-07-11,"2200 West 113th Street, Chicago, IL 60643, USA",CULEX PIPIENS/RESTUANS,22,W 113TH ST,T086,"2200 W 113TH ST, Chicago, IL",41.688324,-87.676709,8,50,0,2007,7,11
351,2007-07-11,"3500 West 116th Street, Chicago, IL 60655, USA",CULEX PIPIENS/RESTUANS,35,W 116TH ST,T158,"3500 W 116TH ST, Chicago, IL",41.682587,-87.707973,9,50,0,2007,7,11
353,2007-07-11,"3500 West 116th Street, Chicago, IL 60655, USA",CULEX PIPIENS/RESTUANS,35,W 116TH ST,T158,"3500 W 116TH ST, Chicago, IL",41.682587,-87.707973,9,50,0,2007,7,11
512,2007-07-18,"3300 West Randolph Street, Chicago, IL 60612, USA",CULEX RESTUANS,33,E RANDOLPH ST,T044,"3300 E RANDOLPH ST, Chicago, IL",41.883284,-87.705085,8,1,0,2007,7,18
531,2007-07-18,"South Stony Island Avenue, Chicago, IL, USA",CULEX PIPIENS/RESTUANS,10,S STONY ISLAND AVE,T138,"1000 S STONY ISLAND AVE, Chicago, IL",41.726465,-87.585413,5,50,0,2007,7,18
548,2007-07-18,"3700 118th Street, Chicago, IL 60617, USA",CULEX PIPIENS/RESTUANS,37,E 118TH ST,T212,"3700 E 118TH ST, Chicago, IL",41.680946,-87.535198,8,50,0,2007,7,18
587,2007-07-19,"3700 118th Street, Chicago, IL 60617, USA",CULEX PIPIENS/RESTUANS,37,E 118TH ST,T212,"3700 E 118TH ST, Chicago, IL",41.680946,-87.535198,8,50,0,2007,7,19
588,2007-07-19,"3700 118th Street, Chicago, IL 60617, USA",CULEX PIPIENS/RESTUANS,37,E 118TH ST,T212,"3700 E 118TH ST, Chicago, IL",41.680946,-87.535198,8,50,0,2007,7,19
589,2007-07-19,"3700 118th Street, Chicago, IL 60617, USA",CULEX PIPIENS/RESTUANS,37,E 118TH ST,T212,"3700 E 118TH ST, Chicago, IL",41.680946,-87.535198,8,50,0,2007,7,19


In [37]:
test.head(1)

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,Year,Month,Day
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,2008,6,11


In [38]:
test.describe()

,Id,Block,Latitude,Longitude,AddressAccuracy,Year,Month,Day
count,116293.000000,116293.000000,116293.000000,116293.000000,116293.000000,116293.000000,116293.000000,116293.000000
mean,58147.000000,41.131100,41.849389,-87.693658,7.954357,2010.702759,7.620244,16.187681
std,33571.041765,24.864726,0.106593,0.080699,1.252733,2.128750,1.065842,8.800007
min,1.000000,10.000000,41.644612,-87.930995,3.000000,2008.000000,6.000000,1.000000
25%,29074.000000,18.000000,41.753411,-87.750938,8.000000,2008.000000,7.000000,10.000000
50%,58147.000000,39.000000,41.862292,-87.694991,8.000000,2010.000000,8.000000,17.000000
75%,87220.000000,61.000000,41.951866,-87.648860,9.000000,2012.000000,8.000000,24.000000
max,116293.000000,98.000000,42.017430,-87.531635,9.000000,2014.000000,10.000000,31.000000


In [39]:
print("# of unique train set species: ", train['Species'].nunique(), '\n')
train['Species'].value_counts()

# of unique train set species:  7 



CULEX PIPIENS/RESTUANS    4752
CULEX RESTUANS            2740
CULEX PIPIENS             2699
CULEX TERRITANS            222
CULEX SALINARIUS            86
CULEX TARSALIS               6
CULEX ERRATICUS              1
Name: Species, dtype: int64

In [40]:
# Checking which species have the West Nile virus
train.groupby('WnvPresent')['Species'].value_counts()

WnvPresent  Species               
0           CULEX PIPIENS/RESTUANS    4490
            CULEX RESTUANS            2691
            CULEX PIPIENS             2459
            CULEX TERRITANS            222
            CULEX SALINARIUS            86
            CULEX TARSALIS               6
            CULEX ERRATICUS              1
1           CULEX PIPIENS/RESTUANS     262
            CULEX PIPIENS              240
            CULEX RESTUANS              49
Name: Species, dtype: int64

In [41]:
train.WnvPresent.value_counts()

0    9955
1     551
Name: WnvPresent, dtype: int64

In [42]:
print("# of unique test set species: ", test['Species'].nunique(), '\n')
test['Species'].value_counts()

# of unique test set species:  8 



CULEX PIPIENS/RESTUANS    15359
CULEX RESTUANS            14670
CULEX PIPIENS             14521
CULEX SALINARIUS          14355
CULEX TERRITANS           14351
CULEX TARSALIS            14347
CULEX ERRATICUS           14345
UNSPECIFIED CULEX         14345
Name: Species, dtype: int64

In [43]:
print("# of unique train set traps: ", train['Trap'].nunique(), '\n')
train['Trap'].value_counts().head()

# of unique train set traps:  136 



T900    750
T115    542
T138    314
T002    185
T135    183
Name: Trap, dtype: int64

In [44]:
# We have 138 locations for the traps but we have 136 traps 
# which means there are 2 traps have been moved to new locations

In [45]:
train.groupby("Trap")["AddressNumberAndStreet"].nunique().value_counts()

1    134
2      2
Name: AddressNumberAndStreet, dtype: int64

In [46]:
train.groupby("Trap")["AddressNumberAndStreet"].nunique() == 2

Trap
T001    False
T002    False
T003    False
T004    False
T005    False
T006    False
T007    False
T008    False
T009     True
T011    False
T012    False
T013    False
T014    False
T015    False
T016    False
T017    False
T018    False
T019    False
T025    False
T027    False
T028    False
T030    False
T031    False
T033    False
T034    False
T035     True
T036    False
T037    False
T039    False
T040    False
        ...  
T160    False
T161    False
T162    False
T200    False
T206    False
T209    False
T212    False
T215    False
T218    False
T219    False
T220    False
T221    False
T222    False
T223    False
T224    False
T225    False
T226    False
T227    False
T228    False
T229    False
T230    False
T231    False
T232    False
T233    False
T235    False
T236    False
T237    False
T238    False
T900    False
T903    False
Name: AddressNumberAndStreet, Length: 136, dtype: bool

In [47]:
# T009 and T035 have been moved from thier locations

In [48]:
train[train["Trap"] == "T035"]

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Year,Month,Day
389,2007-07-11,"3000 South Hoyne Avenue, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,30,S HOYNE AVE,T035,"3000 S HOYNE AVE, Chicago, IL",41.836644,-87.677737,8,1,0,2007,7,11
513,2007-07-18,"3000 South Hoyne Avenue, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,30,S HOYNE AVE,T035,"3000 S HOYNE AVE, Chicago, IL",41.836644,-87.677737,8,1,0,2007,7,18
561,2007-07-19,"3000 South Hoyne Avenue, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,30,S HOYNE AVE,T035,"3000 S HOYNE AVE, Chicago, IL",41.836644,-87.677737,8,3,0,2007,7,19
759,2007-07-27,"3000 South Hoyne Avenue, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,30,S HOYNE AVE,T035,"3000 S HOYNE AVE, Chicago, IL",41.836644,-87.677737,8,3,0,2007,7,27
760,2007-07-27,"3000 South Hoyne Avenue, Chicago, IL 60608, USA",CULEX RESTUANS,30,S HOYNE AVE,T035,"3000 S HOYNE AVE, Chicago, IL",41.836644,-87.677737,8,1,0,2007,7,27
1061,2007-08-01,"3000 South Hoyne Avenue, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,30,S HOYNE AVE,T035,"3000 S HOYNE AVE, Chicago, IL",41.836644,-87.677737,8,6,0,2007,8,1
1062,2007-08-01,"3000 South Hoyne Avenue, Chicago, IL 60608, USA",CULEX PIPIENS,30,S HOYNE AVE,T035,"3000 S HOYNE AVE, Chicago, IL",41.836644,-87.677737,8,5,0,2007,8,1
1602,2007-08-07,"3000 South Hoyne Avenue, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,30,S HOYNE AVE,T035,"3000 S HOYNE AVE, Chicago, IL",41.836644,-87.677737,8,4,0,2007,8,7
1603,2007-08-07,"3000 South Hoyne Avenue, Chicago, IL 60608, USA",CULEX RESTUANS,30,S HOYNE AVE,T035,"3000 S HOYNE AVE, Chicago, IL",41.836644,-87.677737,8,1,0,2007,8,7
1954,2007-08-15,"3000 South Hoyne Avenue, Chicago, IL 60608, USA",CULEX PIPIENS/RESTUANS,30,S HOYNE AVE,T035,"3000 S HOYNE AVE, Chicago, IL",41.836644,-87.677737,8,9,0,2007,8,15


In [49]:
train[train["Trap"]== "T009"]

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Year,Month,Day
50,2007-06-05,"9100 West Higgins Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,91,W HIGGINS AVE,T009,"9100 W HIGGINS AVE, Chicago, IL",41.981964,-87.812827,8,2,0,2007,6,5
221,2007-07-02,"9100 West Higgins Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,91,W HIGGINS AVE,T009,"9100 W HIGGINS AVE, Chicago, IL",41.981964,-87.812827,8,2,0,2007,7,2
312,2007-07-11,"9100 West Higgins Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,91,W HIGGINS AVE,T009,"9100 W HIGGINS AVE, Chicago, IL",41.981964,-87.812827,8,1,0,2007,7,11
313,2007-07-11,"9100 West Higgins Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,91,W HIGGINS AVE,T009,"9100 W HIGGINS AVE, Chicago, IL",41.981964,-87.812827,8,14,0,2007,7,11
714,2007-07-27,"9100 West Higgins Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,91,W HIGGINS AVE,T009,"9100 W HIGGINS AVE, Chicago, IL",41.981964,-87.812827,8,1,0,2007,7,27
715,2007-07-27,"9100 West Higgins Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,91,W HIGGINS AVE,T009,"9100 W HIGGINS AVE, Chicago, IL",41.981964,-87.812827,8,1,0,2007,7,27
716,2007-07-27,"9100 West Higgins Avenue, Chicago, IL 60656, USA",CULEX PIPIENS,91,W HIGGINS AVE,T009,"9100 W HIGGINS AVE, Chicago, IL",41.981964,-87.812827,8,1,0,2007,7,27
862,2007-08-01,"9100 West Higgins Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,91,W HIGGINS AVE,T009,"9100 W HIGGINS AVE, Chicago, IL",41.981964,-87.812827,8,2,0,2007,8,1
863,2007-08-01,"9100 West Higgins Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,91,W HIGGINS AVE,T009,"9100 W HIGGINS AVE, Chicago, IL",41.981964,-87.812827,8,6,0,2007,8,1
864,2007-08-01,"9100 West Higgins Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,91,W HIGGINS AVE,T009,"9100 W HIGGINS AVE, Chicago, IL",41.981964,-87.812827,8,2,0,2007,8,1


In [50]:
print("# of unique test set traps: ", test['Trap'].nunique(), '\n')
test['Trap'].value_counts().head()

# of unique test set traps:  149 



T009    1528
T035    1520
T900    1468
T002     857
T008     822
Name: Trap, dtype: int64

In [51]:
# Drop address Columns

train = train.drop(['Address','AddressNumberAndStreet', "Block", "Street", "AddressAccuracy"], axis=1)
test = test.drop(['Id', 'Address','AddressNumberAndStreet', "Block", "Street", "AddressAccuracy"], axis=1)

In [52]:
# to make Trap into numerical columns

train['Trap'] = train['Trap'].str.strip('TABCabc').astype(int)
test['Trap'] = test['Trap'].str.strip('TABCabc').astype(int)

##### Spray dataset (not much to use it for the EDA and modeling)

In [53]:
spray.head().T

,0,1,2,3,4
Date,2011-08-29,2011-08-29,2011-08-29,2011-08-29,2011-08-29
Time,6:56:58 PM,6:57:08 PM,6:57:18 PM,6:57:28 PM,6:57:38 PM
Latitude,42.3916,42.3913,42.391,42.3906,42.3904
Longitude,-88.0892,-88.0892,-88.0892,-88.0892,-88.0889
Year,2011,2011,2011,2011,2011
Month,8,8,8,8,8
Day,29,29,29,29,29


In [54]:
spray.describe()

,Latitude,Longitude,Year,Month,Day
count,14835.000000,14835.000000,14835.000000,14835.000000,14835.000000
mean,41.904828,-87.736690,2012.702191,7.948028,17.073812
std,0.104381,0.067292,0.712013,0.677402,7.959018
min,41.713925,-88.096468,2011.000000,7.000000,5.000000
25%,41.785001,-87.794225,2013.000000,7.000000,8.000000
50%,41.940075,-87.727853,2013.000000,8.000000,17.000000
75%,41.980978,-87.694108,2013.000000,8.000000,25.000000
max,42.395983,-87.586727,2013.000000,9.000000,29.000000


##### Drop? / Duplicate
- Spray Time duplication - "7:44:32 PM", 541 counts
- Spray Time NaN values, 584 counts

In [55]:
# weird. How 7:44:32 pm has 541 values? are they using auto-spray function?

spray.Time.value_counts().head()

7:44:32 PM    541
8:55:46 PM      5
8:59:06 PM      5
8:58:56 PM      5
9:37:27 PM      5
Name: Time, dtype: int64

In [56]:
# it seems the duplicated data. However do we need to drop it? 
spray[spray.Time == "7:44:32 PM"].head()

,Date,Time,Latitude,Longitude,Year,Month,Day
489,2011-09-07,7:44:32 PM,41.98646,-87.794225,2011,9,7
490,2011-09-07,7:44:32 PM,41.98646,-87.794225,2011,9,7
491,2011-09-07,7:44:32 PM,41.98646,-87.794225,2011,9,7
492,2011-09-07,7:44:32 PM,41.98646,-87.794225,2011,9,7
493,2011-09-07,7:44:32 PM,41.98646,-87.794225,2011,9,7


In [57]:
spray[spray['Time'].isnull()]

,Date,Time,Latitude,Longitude,Year,Month,Day
1030,2011-09-07,NaN,41.987092,-87.794286,2011,9,7
1031,2011-09-07,NaN,41.987620,-87.794382,2011,9,7
1032,2011-09-07,NaN,41.988004,-87.794574,2011,9,7
1033,2011-09-07,NaN,41.988292,-87.795486,2011,9,7
1034,2011-09-07,NaN,41.988100,-87.796014,2011,9,7
1035,2011-09-07,NaN,41.986372,-87.794862,2011,9,7
1036,2011-09-07,NaN,41.986228,-87.795582,2011,9,7
1037,2011-09-07,NaN,41.984836,-87.793998,2011,9,7
1038,2011-09-07,NaN,41.984836,-87.794670,2011,9,7
1039,2011-09-07,NaN,41.984884,-87.795198,2011,9,7


In [58]:
spray[spray.Longitude == -87.794225]['Time'].value_counts()

7:44:32 PM    541
Name: Time, dtype: int64

**Merge with weather Data**

In [59]:
train = train.merge(weather, on='Date')
test = test.merge(weather, on='Date')
train = train.drop(['Date'], axis = 1)
test = test.drop(['Date'], axis = 1)

Let's make it binary columns for the type of mosquitos

In [60]:
#courtesy of Brian Collins

def bug_find(df):
    for bug in df:
        if bug == 'CULEX PIPIENS/RESTUANS':
            return 'wnv_bug'
        elif bug == 'CULEX PIPIENS':
            return 'wnv_bug'
        elif bug == 'CULEX RESTUANS':
            return 'wnv_bug'
        elif bug == 'UNSPECIFIED CULEX':
            return 'wnv_bug'
        else:
            return 'no_wnv_bug'

train['Species'] = train[['Species']].apply(bug_find, axis=1)
test['Species'] = test[['Species']].apply(bug_find, axis=1)
print(train['Species'].value_counts())
print(test['Species'].value_counts())

wnv_bug       10191
no_wnv_bug      315
Name: Species, dtype: int64
wnv_bug       58895
no_wnv_bug    57398
Name: Species, dtype: int64


In [61]:
test['Species'].value_counts()

wnv_bug       58895
no_wnv_bug    57398
Name: Species, dtype: int64

In [62]:
# Converting all the objects in train data to floates or integers
train["Depart_x"] = train["Depart_x"].astype(float)

train["WetBulb_x"] = train["WetBulb_x"].astype(float)

train["Sunset_x"] = train["Sunset_x"].astype(float)

train["Sunrise_x"] = train["Sunrise_x"].astype(float)

train["WetBulb_y"] = train["WetBulb_y"].astype(float)

# train = pd.get_dummies(train, columns=["CodeSum_x", "CodeSum_y"])

In [63]:
# Converting all the objects in test data to floates or integers
test["Depart_x"] = test["Depart_x"].astype(float)

test["WetBulb_x"] = test["WetBulb_x"].astype(float)

test["Sunset_x"] = test["Sunset_x"].astype(float)

test["Sunrise_x"] = test["Sunrise_x"].astype(float)

test["WetBulb_y"] = test["WetBulb_y"].astype(float)

# test = pd.get_dummies(test, columns=["CodeSum_x", "CodeSum_y"])

In [64]:
train = train.drop(columns=["CodeSum_x","CodeSum_y"])
test = test.drop(columns=["CodeSum_x","CodeSum_y"])

### Let's start modeling

In [65]:
train = pd.get_dummies(train, columns=['Species'], drop_first=True)
test = pd.get_dummies(test, columns=['Species'], drop_first=True)

In [66]:
print(train.shape, test.shape)

(10506, 41) (116293, 39)


In [67]:
train.columns

Index(['Trap', 'Latitude', 'Longitude', 'NumMosquitos', 'WnvPresent', 'Year_x',
       'Month_x', 'Day_x', 'Tmax_x', 'Tmin_x', 'Tavg_x', 'Depart_x',
       'DewPoint_x', 'WetBulb_x', 'Heat_x', 'Cool_x', 'Sunrise_x', 'Sunset_x',
       'PrecipTotal_x', 'StnPressure_x', 'SeaLevel_x', 'ResultSpeed_x',
       'ResultDir_x', 'AvgSpeed_x', 'Tmax_y', 'Tmin_y', 'Tavg_y', 'DewPoint_y',
       'WetBulb_y', 'Heat_y', 'Cool_y', 'PrecipTotal_y', 'StnPressure_y',
       'SeaLevel_y', 'ResultSpeed_y', 'ResultDir_y', 'AvgSpeed_y', 'Year_y',
       'Month_y', 'Day_y', 'Species_wnv_bug'],
      dtype='object')

In [68]:
test.columns

Index(['Trap', 'Latitude', 'Longitude', 'Year_x', 'Month_x', 'Day_x', 'Tmax_x',
       'Tmin_x', 'Tavg_x', 'Depart_x', 'DewPoint_x', 'WetBulb_x', 'Heat_x',
       'Cool_x', 'Sunrise_x', 'Sunset_x', 'PrecipTotal_x', 'StnPressure_x',
       'SeaLevel_x', 'ResultSpeed_x', 'ResultDir_x', 'AvgSpeed_x', 'Tmax_y',
       'Tmin_y', 'Tavg_y', 'DewPoint_y', 'WetBulb_y', 'Heat_y', 'Cool_y',
       'PrecipTotal_y', 'StnPressure_y', 'SeaLevel_y', 'ResultSpeed_y',
       'ResultDir_y', 'AvgSpeed_y', 'Year_y', 'Month_y', 'Day_y',
       'Species_wnv_bug'],
      dtype='object')

In [69]:
target = train.WnvPresent
features = train.drop(columns=['WnvPresent', "NumMosquitos"])

In [70]:
X_train, X_test, y_train, y_test = train_test_split(features,
                                                    target,
                                                    stratify=target,
                                                    random_state=42)


In [71]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

### modeling

In [72]:
# lr = LogisticRegression()
# rf = RandomForestClassifier()
# ada = AdaBoostClassifier()
# pipe = make_pipeline()

In [73]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
cross_val_score(lr, X_train, y_train).mean()

0.9475822381737875

In [74]:
cross_val_score(lr, X_test, y_test).mean()

0.9474685801261143

In [75]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
cross_val_score(rf, X_train, y_train).mean()

0.9361592739620909

In [76]:
cross_val_score(rf, X_test, y_test).mean()

0.9326205696890627

In [77]:
ada = AdaBoostClassifier()
ada.fit(X_train, y_train)
cross_val_score(ada, X_train, y_train).mean()

0.947074398342004

In [78]:
cross_val_score(ada, X_test, y_test).mean()

0.9440417482061317

In [97]:
rf.get_params

<bound method BaseEstimator.get_params of RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)>

### Let's do some GridSearch

In [79]:
lr_params = {
    'penalty' : ['l1', 'l2']
}

gs = GridSearchCV(lr, param_grid=lr_params)
gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

0.9475821804797563
{'penalty': 'l1'}


In [80]:
gs.score(X_test, y_test)

0.9474685953559193

In [81]:
print(gs.best_estimator_)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [82]:
X_train.shape

(7879, 39)

In [83]:
test = ss.transform(test)

In [84]:
pred = gs.predict_proba(test)[:,1]

In [85]:
pred = pd.DataFrame(pred)

In [86]:
pred["WnvPresent"] = pred

In [87]:
pred.drop(columns=[0],inplace=True)

In [88]:
pred["Id"]= pred.index

In [89]:
pred["Id"] += 1

In [90]:
pred = pred[["Id","WnvPresent"]]

In [91]:
# pred.to_csv("pred.csv", index=False)

In [92]:
pred["WnvPresent"].value_counts()

0.199474    37
0.013131    35
0.131098    32
0.002850    23
0.302122    22
0.214008    21
0.100531    21
0.034400    20
0.087461    20
0.282149    20
0.170114    18
0.095074    18
0.034221    18
0.038375    18
0.304480    18
0.015946    17
0.508534    17
0.116423    17
0.620444    17
0.066876    17
0.003438    17
0.139036    17
0.438997    17
0.139971    16
0.180586    16
0.013881    16
0.422738    16
0.025047    16
0.159647    15
0.000786    15
            ..
0.000924     4
0.055596     4
0.063956     4
0.008503     4
0.000007     4
0.000454     4
0.000278     4
0.000437     4
0.000058     4
0.005320     4
0.056896     4
0.005691     4
0.109059     4
0.000029     4
0.002772     4
0.000029     4
0.002377     4
0.028671     4
0.000462     4
0.000140     4
0.018590     4
0.000930     4
0.001639     4
0.003835     4
0.000085     4
0.000029     4
0.090613     4
0.000005     4
0.012790     4
0.048017     4
Name: WnvPresent, Length: 28690, dtype: int64

In [83]:
test = ss.transform(test)

In [84]:
pred = gs.predict_proba(test)[:,1]

In [85]:
pred = pd.DataFrame(pred)

In [86]:
pred["WnvPresent"] = pred

In [87]:
pred.drop(columns=[0],inplace=True)

In [88]:
pred["Id"]= pred.index

In [89]:
pred["Id"] += 1

In [90]:
pred = pred[["Id","WnvPresent"]]

In [91]:
# pred.to_csv("pred.csv", index=False)

In [92]:
pred["WnvPresent"].value_counts()

0.199474    37
0.013131    35
0.131098    32
0.002850    23
0.302122    22
0.214008    21
0.100531    21
0.034400    20
0.087461    20
0.282149    20
0.170114    18
0.095074    18
0.034221    18
0.038375    18
0.304480    18
0.015946    17
0.508534    17
0.116423    17
0.620444    17
0.066876    17
0.003438    17
0.139036    17
0.438997    17
0.139971    16
0.180586    16
0.013881    16
0.422738    16
0.025047    16
0.159647    15
0.000786    15
            ..
0.000924     4
0.055596     4
0.063956     4
0.008503     4
0.000007     4
0.000454     4
0.000278     4
0.000437     4
0.000058     4
0.005320     4
0.056896     4
0.005691     4
0.109059     4
0.000029     4
0.002772     4
0.000029     4
0.002377     4
0.028671     4
0.000462     4
0.000140     4
0.018590     4
0.000930     4
0.001639     4
0.003835     4
0.000085     4
0.000029     4
0.090613     4
0.000005     4
0.012790     4
0.048017     4
Name: WnvPresent, Length: 28690, dtype: int64

**Let's find some plotting**

In [93]:
# trying to find some plotting here

# plt.plot(train['WnvPresent'], train['Species'], 'r')

##### Check numerical columns data, find abnormalty or outliers

In [94]:
print(train.isnull().sum().sum())
print(weather.isnull().sum().sum())
print(spray.isnull().sum().sum())


0
0
584


In [95]:
# import all relevant packages

import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler

from sklearn.cross_validation import train_test_split

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import Dropout

%matplotlib inline

ModuleNotFoundError: No module named 'keras'

In [98]:
pandas_profiling.ProfileReport(train)

Number of variables,41
Number of observations,10506
Total Missing (%),0.0%
Total size in memory,3.6 MiB
Average record size in memory,360.2 B
Numeric,22
Categorical,0
Boolean,2
Date,0
Text (Unique),0
Rejected,17


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
train.groupby('Year')['NumMosquitos'].sum().plot(kind='bar')
plt.ylabel('Number of Mosquitos')
plt.title('Number of Mosquitos Per Year');

In [ ]:
train.groupby('Month')['NumMosquitos'].mean().plot(kind='bar')
plt.ylabel('Average Number of Mosquitos')
plt.title('Average Number of Mosquitos Per Month');

In [ ]:
train.groupby('Tavg_x')['NumMosquitos'].mean().plot(kind='bar')
plt.ylabel('Average Number of Mosquitos')
plt.xlabel('Average Temperature (Farenheit)')
plt.title('Average Number of Mosquitos Depending on Average Temperature');